# Visualize Landprices


In [1]:
import pandas as pd
import geopandas as gpd
import os

C:\Users\BerkÖztürk\AppData\Local\Temp\ipykernel_7964\1734800707.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


### Vector data 
We start by reading and quickly visualising vector data

### Read raster data

We read the census data that is classified in a grid, then we read the grid as a geopandas file.

In [2]:
ZENSUS_PATH = r"../data/raw/2 Zensus/"
NEIGHBORHOOD_PATH =  r"../data/raw/3 Neighborhoods"
LAND_PRICES_PATH = r"../data/raw/1 Land Prices"
CITY_NAMES=["Berlin","Bremen","Dresden","Frankfurt","Köln"]
zensus_files= os.listdir(ZENSUS_PATH)
neighborhood_files= os.listdir(NEIGHBORHOOD_PATH)
landprices_files= os.listdir(LAND_PRICES_PATH)

def get_file_names(): 
    city_files={}
    for city_name in CITY_NAMES:
        
        csv_fpaths= [os.path.join(ZENSUS_PATH,fpath) for fpath in zensus_files if ( city_name in fpath ) and (fpath.endswith(".csv")) ]
        gpkg_fpaths= [os.path.join(ZENSUS_PATH,fpath) for fpath in zensus_files if ( city_name in fpath ) and (fpath.endswith(".gpkg")) ]
        
        neighbourhood_fpaths= [os.path.join(NEIGHBORHOOD_PATH,fpath) for fpath in neighborhood_files if ( city_name in fpath ) and (fpath.endswith(".gpkg")) ]
        landprices_fpaths= [os.path.join(LAND_PRICES_PATH,fpath) for fpath in landprices_files if ( city_name in fpath ) ]

        city_files[city_name]= (csv_fpaths,gpkg_fpaths,neighbourhood_fpaths,landprices_fpaths)
    return city_files

def combine_data_within_city(city_files):
    city_merged_data={}
    for city_name, (csv_fpaths,gpkg_fpaths,neighbourhood_fpaths,landprices_fpaths) in city_files.items():
        df_list_city=[]
        for csv_fpath in csv_fpaths:
            df = pd.read_csv(csv_fpath, sep=";", encoding="utf-8-sig").drop(columns="Unnamed: 0")
            df_list_city.append(df)

        from functools import reduce
        zensus_csv_merged = reduce(lambda df1,df2: pd.merge(df1,df2,on="Grid_Code"), df_list_city)
        
        # read grid
        grid_city = gpd.read_file(gpkg_fpaths[0])
        grid_city = grid_city.merge(zensus_csv_merged, on = "Grid_Code")

        # merge files
        prices_city = pd.read_csv(landprices_fpaths[0], sep=";", encoding= "utf-8-sig").drop(columns="Unnamed: 0")
        neighborhood_city = gpd.read_file(neighbourhood_fpaths[0])
        neighborhood_city = neighborhood_city.merge(prices_city, on = "Neighborhood_FID", how = "inner")

        # Perform spatial join using sjoin
        merged_data = gpd.sjoin(grid_city, neighborhood_city, how='left', op='intersects')
        merged_data = merged_data.drop(columns=["City_Name_y", "City_Code_right"]).rename(columns={"City_Name_x":"City_Name", "City_Code_left":"City_Code"})
        city_merged_data[city_name] = merged_data
    return city_merged_data

def concat_city_dataframes(city_merged_data):
    col_list=[]
    for city_name,df in city_merged_data.items():
        print(df.shape)
        col_list.append(df.columns.to_list())
    common_cols=list(set(col_list[0]).intersection(*col_list))

    grid_level_df_list = [df[common_cols] for df in city_merged_data.values()]
    grid_level_all_cities = pd.concat(grid_level_df_list, axis=0)
    return grid_level_all_cities

In [3]:
city_files= get_file_names()


city_merged_data = combine_data_within_city(city_files)



grid_level_all_cities= concat_city_dataframes(city_merged_data)


c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and

(96838, 194)
(35637, 196)
(37357, 194)
(27329, 193)
(45330, 194)


In [11]:
(grid_level_all_cities.fams_seniors_young_x== grid_level_all_cities.fams_seniors_young_y).mean()

0.8266574841952897

In [18]:
def duplicate_columns(frame):
    groups = frame.columns.to_series().groupby(frame.dtypes).groups
    dups = []
    for t, v in groups.items():
        dcols = frame[v].to_dict(orient="list")

        vs = dcols.values()
        ks = dcols.keys()
        lvs = len(vs)

        for i in range(lvs):
            for j in range(i+1,lvs):
                if vs[i] == vs[j]: 
                    dups.append(ks[i])
                    break

    return dups       

In [36]:
# Save merged data to file
grid_level_all_cities.to_file('../data/interim/grid_level_merged_all_cities.gpkg', driver='GPKG')